In [11]:
# Example and metrics from paper: 
# FAIRNESS METRICS : A COMPARATIVE ANALYSIS
# https://arxiv.org/abs/2001.07864

from outcomes import Outcomes
from fairnessMetrics import FairnessMetrics 

outcomes1 = Outcomes(28,12,12,8)
outcomes2 = Outcomes(8,16,12,4)

print(FairnessMetrics.equalizedOdds(outcomes1))
print(FairnessMetrics.equalizedOdds(outcomes2))
print(FairnessMetrics.equalizedOddsDiff(outcomes1, outcomes2))

print(FairnessMetrics.statisticalParity(outcomes1))
print(FairnessMetrics.statisticalParity(outcomes2))
print(FairnessMetrics.statisticalParityDiff(outcomes1, outcomes2))

print(FairnessMetrics.predictiveParity(outcomes1))
print(FairnessMetrics.predictiveParity(outcomes2))
print(FairnessMetrics.predictiveParityDiff(outcomes1, outcomes2))

(0.7, 0.4)
(0.4, 0.2)
(0.29999999999999993, 0.2)
0.6
0.3
0.3
0.7777777777777778
0.6666666666666666
0.11111111111111116
